In [1]:
from extract_utils import *

# Pre-Processing Outline

1. Epochs for selected region (window=5s)
2. Transform raw data to frequency domain, look for peaks at 0.4-5 Hz
3. outliners identification -- e.g. 95% percentile
4. Filter out epochs based on:
    - peaks at 0.4-5 Hz
    - within 95% percentile of power? 


## Obtain relevant patient info

- Here we choose patient 8 for our analysis
- Figure shown below indicidates the position of electrodes on this patient

![elecLoc](Data/electrodePos/ep_sbj_08.png)

In [2]:
# get the nwb file
file, _ = get_nwb('08', 3)

/Applications/anaconda3/envs/NMA/lib/python3.9/site-packages/hdmf/spec/namespace.py:531: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.4.0-alpha because version 1.7.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/Applications/anaconda3/envs/NMA/lib/python3.9/site-packages/hdmf/spec/namespace.py:531: UserWarning: Ignoring cached namespace 'core' version 2.2.5 because version 2.6.0-alpha is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/Applications/anaconda3/envs/NMA/lib/python3.9/site-packages/hdmf/spec/namespace.py:531: UserWarning: Ignoring cached namespace 'hdmf-experimental' version 0.1.0 because version 0.4.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."


In [3]:
# table of epochs for all behavioural type
allActs, blocklist, acts, epochs = get_epochs(file, True)

# get raw data
raw = file.acquisition['ElectricalSeries'].data
# get good electrode id defined in the literature
goodid = file.electrodes.to_dataframe()[file.electrodes.to_dataframe()['good']==True].index

## Obtain Epochs for each behaviors

In [4]:
# extract all inactive activities
inactive = extract_phys_accod_timestamps(raw, epochs['Inactive'], [0, 24])
# use the average as baseline
d = []
for item in inactive.keys():
    d.append(epoching(inactive[item])[:, goodid.values,:]) # leave only good electrodes
inactiveArray = np.dstack(d)
inactiveArray.shape
baseline = inactiveArray.mean(axis=2)

In [15]:
epochs_behavior = {} # Mapping behavior to corresponding epochs of 5s

for item in acts:
    # extract recording by bahvior
    if item == 'Inactive' or item=='Other activity' or item=='Sleep/rest':
        pass
    else:
        temp = extract_phys_accod_timestamps(raw, epochs[item], [0, 24])
        # obtain epochs for each behaviour
        d = []
        for key in temp.keys():
            d.append(epoching(temp[key])[:, goodid.values,:]) # leave only good electrodes
        epochs_behavior[item] = np.dstack(d)

In [72]:
acts

['Computer/phone',
 'Eat, TV',
 'Inactive',
 'Other activity',
 'Sleep/rest',
 'TV',
 'TV, Computer/phone',
 'Talk',
 'Talk, Computer/phone',
 'Talk, TV',
 'Talk, TV, Computer/phone']

## Power Transform

In [52]:
import fooof as f
import scipy
from scipy.signal import welch, firwin, fftconvolve, hilbert
from scipy.fft import ifft
from scipy.signal import sosfilt

In [83]:
acts = ['Computer/phone',
 'Eat, TV',
 'TV',
 'TV, Computer/phone',
 'Talk',
 'Talk, Computer/phone',
 'Talk, TV',
 'Talk, TV, Computer/phone']

In [150]:
# 4th order IIR filter to extract low-frequence envelop
filter_order = 4
neural_freq_range = [0.5, 5]
sampling_rate = 500
filtered_data = {}
for item in acts:
        item_specific = np.empty((epochs_behavior[item].shape))
        for j in range(epochs_behavior[item].shape[1]):
                neur_data_curr = epochs_behavior[item][:, j, :]
                sos = butter(filter_order, neural_freq_range, btype='bandpass', output='sos', fs=sampling_rate)
                item_specific[:, j, :] = sosfiltfilt(sos, neur_data_curr, padlen=0)
        filtered_data[item] = item_specific

In [151]:
# Select epochs with power within 99% percentile

for j in range(filtered_data['TV'].shape[1]): # for each channel
    channel_curr = []
    for i in range(filtered_data['TV'].shape[-1]): # for each epoch
        # Hilbert transform on filtered data
        # power = np.abs(hilbert(filtered_data['TV'][:, j, i], ))
        # channel_curr.append(power)
        # welch transform
        freqs, spectrum = welch(filtered_data['TV'][:, j, i], fs=500, scaling='spectrum')
        
        